In [19]:
import numpy as np
import matplotlib as plot
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [90]:
#preprocessing
df = pd.read_csv('data\Immunisations.csv')
to_drop = ['VAR','UNIT','Measure','COU','YEA', 'Flag Codes','Flags']
df.drop(to_drop, inplace=True, axis=1)

#creating a separate dataframe with just the doctors  
df_doc = pd.read_csv('data\Physicians.csv')
to_drop = ['VAR','UNIT','Measure','COU','YEA', 'Flag Codes','Flags']
df_doc.drop(to_drop, inplace=True, axis=1)
df_doc = df_doc[df_doc['Variable'].str.contains('Practising physicians')]
merged = pd.merge(left=df, right=df_doc, how='right', left_on=['Year','Country'], right_on=['Year','Country'])
merged = merged.rename(columns={'Value_x': 'percent_immun', 'Value_y': 'num_physicians'})
merged = merged.dropna()
merged['num_physicians_sqr'] = merged['num_physicians'] * merged['num_physicians']
column = 'num_physicians'
merged[column] = (merged[column] - merged[column].mean()) / merged[column].std()

#split data
train, test = train_test_split(merged, test_size=0.2)
x_train = train.loc[:, ['num_physicians', 'num_physicians_sqr']]
y_train = train["percent_immun"].to_frame(name = "y_train")
x_test = test.loc[:, ['num_physicians', 'num_physicians_sqr']]
y_test = test["percent_immun"].to_frame(name = "y_test")


In [95]:
model = LinearRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test,y_pred)

In [96]:
r2

0.006658360196676538